In [1]:
from gqimax2.mapper import map_cx, map_noncx
from gqimax2.sample import sample1, sample2
from gqimax2.mapper import map_indices_to_indicess, map_noncx
import cupy as cp
import time
%load_ext autoreload
%autoreload 2
num_qubits = 4
ins = sample1(num_qubits)
ins.operatoring()
from gqimax2.pstabilizers import PStabilizers
stb = PStabilizers(num_qubits)
lambdass = stb.lambdass
indicesss = stb.indicess

In [2]:
lambdass

[array([1.]), array([1.]), array([1.]), array([1.])]

In [3]:
from gqimax2.mapper import broadcasted_multiplies, broadcasted_multiplies_base4

def weightsss_to_lambdass(lambdass, weightsss, indicesss):
    num_qubits = len(weightsss)
    lambdass_res = [None] * num_qubits
    indicess_res = [None] * num_qubits
    for i in range(num_qubits):
        print(lambdass[i])
        lambdass_res[i] = broadcasted_multiplies(lambdass[i], weightsss[i])
        print(indicesss[i])
        indicess_res[i] = broadcasted_multiplies_base4(indicesss[i])
    return lambdass_res, indicess_res

def map_noncx(lambdass, indicesss, lut_at_k):
    weightsss = []
    for indicess in indicesss:
        weightss = []
        for qubit, indices in enumerate(indicess):
            weights = []
            for index in indices:
                if index == 0:
                    weights.append(cp.array([0], dtype=cp.float32))
                else: 
                    weights.append(lut_at_k[qubit][int(index) - 1])
            weightss.append(weights)
        weightsss.append(weightss)

    return weightsss_to_lambdass(lambdass, weightsss, indicesss)
for j, order in enumerate(ins.orders):
    k = j // 2
    if order == 0:
        lambdass, indicesss = map_noncx(lambdass, indicesss, ins.lut[k])

    else:
        for _, indices, _ in ins.xoperators[k]:
            lambdass, indicesss = map_cx(lambdass, indicesss, indices[0], indices[1])

C:\Users\haime\AppData\Local\Temp\ipykernel_10648\3766601348.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  weights.append(lut_at_k[qubit][int(index) - 1])


[1.]
[[array([3]), array([0]), array([0]), array([0])]]
[1.]
[[array([0]), array([3]), array([0]), array([0])]]
[1.]
[[array([0]), array([0]), array([3]), array([0])]]
[1.]
[[array([0]), array([0]), array([0]), array([3])]]


TypeError: Unsupported type <class 'list'>

In [21]:
ins.lut

[[[array([1.], dtype=float32),
   array([1.], dtype=float32),
   array([1.], dtype=float32)],
  [array([1.], dtype=float32),
   array([0.7109136 , 0.70327944], dtype=float32),
   array([-0.70327944,  0.7109136 ], dtype=float32)],
  [array([1.], dtype=float32),
   array([-1.], dtype=float32),
   array([1.], dtype=float32)],
  [array([1.], dtype=float32),
   array([-1.], dtype=float32),
   array([1.], dtype=float32)]],
 [[array([ 0.8472551, -0.5311862], dtype=float32),
   array([1.], dtype=float32),
   array([0.5311862, 0.8472551], dtype=float32)],
  [array([1.], dtype=float32),
   array([1.], dtype=float32),
   array([1.], dtype=float32)],
  [array([1.], dtype=float32),
   array([1.], dtype=float32),
   array([1.], dtype=float32)],
  [array([1.], dtype=float32),
   array([1.], dtype=float32),
   array([1.], dtype=float32)]]]